In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd


In [7]:
# fetch dataset 
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296) 
  
# data (as pandas dataframes) 
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features 
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets 

C:\Users\aKost\anaconda3\lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [30]:
initial_merge = pd.merge(left=X, right=y, how="outer", left_index=True, right_index=True)
initial_merge.isna().sum()

race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

In [31]:
initial_merge.shape

(101766, 48)

# Dropping columns with more NA values than the race column

In [33]:
threshold = 2273
second_merge = initial_merge.dropna(axis=1, thresh=initial_merge.shape[0] - threshold)
second_merge.isna().sum()

race                        2273
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
num_lab_procedures             0
num_procedures                 0
num_medications                0
number_outpatient              0
number_emergency               0
number_inpatient               0
diag_1                        21
diag_2                       358
diag_3                      1423
number_diagnoses               0
metformin                      0
repaglinide                    0
nateglinide                    0
chlorpropamide                 0
glimepiride                    0
acetohexamide                  0
glipizide                      0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitazone                  0
acarbose                       0
miglitol                       0
troglitazo

In [34]:
second_merge.shape

(101766, 43)

# Looking at unique values of each column

In [35]:
for column in second_merge.columns:
    unique_values = second_merge[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'race': ['Caucasian' 'AfricanAmerican' nan 'Other' 'Asian' 'Hispanic']
Unique values in column 'gender': ['Female' 'Male' 'Unknown/Invalid']
Unique values in column 'age': ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
Unique values in column 'admission_type_id': [6 1 2 3 4 5 8 7]
Unique values in column 'discharge_disposition_id': [25  1  3  6  2  5 11  7 10  4 14 18  8 13 12 16 17 22 23  9 20 15 24 28
 19 27]
Unique values in column 'admission_source_id': [ 1  7  2  4  5  6 20  3 17  8  9 14 10 22 11 25 13]
Unique values in column 'time_in_hospital': [ 1  3  2  4  5 13 12  9  7 10  6 11  8 14]
Unique values in column 'num_lab_procedures': [ 41  59  11  44  51  31  70  73  68  33  47  62  60  55  49  75  45  29
  35  42  66  36  19  64  25  53  52  87  27  37  46  28  48  72  10   2
  65  67  40  54  58  57  43  32  83  34  39  69  38  56  22  96  78  61
  88  50   1  18  82   9  63  24  71  77  81  76  90

# Dropping some columns because their values are not helpful and would be too large for one-hot encoding

In [36]:
second_merge = second_merge.drop(columns=[
    'admission_type_id',
    'discharge_disposition_id',
    'admission_source_id',
    'diag_1',
    'diag_2',
    'diag_3'
])
for column in second_merge.columns:
    unique_values = second_merge[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'race': ['Caucasian' 'AfricanAmerican' nan 'Other' 'Asian' 'Hispanic']
Unique values in column 'gender': ['Female' 'Male' 'Unknown/Invalid']
Unique values in column 'age': ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
Unique values in column 'time_in_hospital': [ 1  3  2  4  5 13 12  9  7 10  6 11  8 14]
Unique values in column 'num_lab_procedures': [ 41  59  11  44  51  31  70  73  68  33  47  62  60  55  49  75  45  29
  35  42  66  36  19  64  25  53  52  87  27  37  46  28  48  72  10   2
  65  67  40  54  58  57  43  32  83  34  39  69  38  56  22  96  78  61
  88  50   1  18  82   9  63  24  71  77  81  76  90  93   3 103  13  80
  85  16  15  12  30  23  17  21  79  26   5  95  97  84  14  74 105  86
  98  20   6  94   8 102 100   7  89  91  92   4 101  99 114 113 111 129
 107 108 106 104 109 120 132 121 126 118]
Unique values in column 'num_procedures': [0 5 1 6 2 3 4]
Unique values in column 'num_

In [38]:
sm_drop_na = second_merge.dropna()

In [39]:
for column in sm_drop_na.columns:
    unique_values = sm_drop_na[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'race': ['Caucasian' 'AfricanAmerican' 'Other' 'Asian' 'Hispanic']
Unique values in column 'gender': ['Female' 'Male' 'Unknown/Invalid']
Unique values in column 'age': ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
Unique values in column 'time_in_hospital': [ 1  3  2  4  5 13 12  9  7 10 11  6  8 14]
Unique values in column 'num_lab_procedures': [ 41  59  11  44  51  31  70  73  68  33  47  62  60  55  49  75  45  29
  35  42  19  64  25  53  52  87  27  37  46  28  36  48  72  10   2  65
  67  40  58  57  32  83  34  39  69  38  56  22  96  78  61  88  66  43
  50   1  18  82  54   9  63  24  71  77  81  76  90  93   3 103  13  80
  85  16  15  12  30  23  17  21  79  26  95  97  84  14  74 105  86  98
  20   5   6  94   8 102 100   7  89  91  92   4 101  99 114 113 111 129
 107 108 106 104 109 120 132 121 126 118]
Unique values in column 'num_procedures': [0 5 1 6 2 3 4]
Unique values in column 'num_medi

In [40]:
sm_drop_na.shape

(99493, 37)

# One-hot encoding

In [41]:
df = sm_drop_na.copy()

columns_to_encode = ['race', 'gender', 'age', 'metformin', 'repaglinide', 'nateglinide', 
                     'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 
                     'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 
                     'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 
                     'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 
                     'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 
                     'diabetesMed', 'readmitted']

for column in columns_to_encode:
    df = pd.concat([df, pd.get_dummies(df[column], prefix=column)], axis=1)
    df.drop(column, axis=1, inplace=True)

for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'time_in_hospital': [ 1  3  2  4  5 13 12  9  7 10 11  6  8 14]
Unique values in column 'num_lab_procedures': [ 41  59  11  44  51  31  70  73  68  33  47  62  60  55  49  75  45  29
  35  42  19  64  25  53  52  87  27  37  46  28  36  48  72  10   2  65
  67  40  58  57  32  83  34  39  69  38  56  22  96  78  61  88  66  43
  50   1  18  82  54   9  63  24  71  77  81  76  90  93   3 103  13  80
  85  16  15  12  30  23  17  21  79  26  95  97  84  14  74 105  86  98
  20   5   6  94   8 102 100   7  89  91  92   4 101  99 114 113 111 129
 107 108 106 104 109 120 132 121 126 118]
Unique values in column 'num_procedures': [0 5 1 6 2 3 4]
Unique values in column 'num_medications': [ 1 18 13 16  8 21 12 28 17 11 15 31  2 23 19  7 20 14 10 22  9 27 25  4
 32  6 30 26 24 33  5 39  3 29 61 40 46 41 36 34 35 50 43 42 37 51 38 45
 54 52 49 62 55 47 44 48 57 53 59 56 63 58 70 67 64 60 69 65 68 66 81 79
 75 72 74]
Unique values in column 'number_outpatient': [ 0  2  1 

In [43]:
df = df.astype(int)
df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,metformin-rosiglitazone_No,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted_<30,readmitted_>30,readmitted_NO
0,1,41,0,1,0,0,0,1,0,0,...,1,1,0,0,1,1,0,0,0,1
1,3,59,0,18,0,0,0,9,0,0,...,1,1,0,1,0,0,1,0,1,0
2,2,11,5,13,2,0,1,6,1,0,...,1,1,0,0,1,0,1,0,0,1
3,2,44,1,16,0,0,0,7,0,0,...,1,1,0,1,0,0,1,0,0,1
4,1,51,0,8,0,0,0,5,0,0,...,1,1,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,3,51,0,16,0,0,0,9,1,0,...,1,1,0,1,0,0,1,0,1,0
101762,5,33,3,18,0,0,1,9,1,0,...,1,1,0,0,1,0,1,0,0,1
101763,1,53,0,9,1,0,0,13,0,0,...,1,1,0,1,0,0,1,0,0,1
101764,10,45,2,21,0,0,1,9,0,0,...,1,1,0,1,0,0,1,0,0,1


In [47]:
for i in [c for c in df.columns]:
    print(i)

time_in_hospital
num_lab_procedures
num_procedures
num_medications
number_outpatient
number_emergency
number_inpatient
number_diagnoses
race_AfricanAmerican
race_Asian
race_Caucasian
race_Hispanic
race_Other
gender_Female
gender_Male
gender_Unknown/Invalid
age_[0-10)
age_[10-20)
age_[20-30)
age_[30-40)
age_[40-50)
age_[50-60)
age_[60-70)
age_[70-80)
age_[80-90)
age_[90-100)
metformin_Down
metformin_No
metformin_Steady
metformin_Up
repaglinide_Down
repaglinide_No
repaglinide_Steady
repaglinide_Up
nateglinide_Down
nateglinide_No
nateglinide_Steady
nateglinide_Up
chlorpropamide_Down
chlorpropamide_No
chlorpropamide_Steady
chlorpropamide_Up
glimepiride_Down
glimepiride_No
glimepiride_Steady
glimepiride_Up
acetohexamide_No
acetohexamide_Steady
glipizide_Down
glipizide_No
glipizide_Steady
glipizide_Up
glyburide_Down
glyburide_No
glyburide_Steady
glyburide_Up
tolbutamide_No
tolbutamide_Steady
pioglitazone_Down
pioglitazone_No
pioglitazone_Steady
pioglitazone_Up
rosiglitazone_Down
rosiglitazon

# Dropping columns ending in "_No" to reduce redundancy 

In [49]:
columns_to_drop = [col for col in df.columns if col.endswith("_No")]
df.drop(columns=columns_to_drop, inplace=True)
columns_to_drop = [col for col in df.columns if col.endswith("_NO")]
df.drop(columns=columns_to_drop, inplace=True)
df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,glyburide-metformin_Down,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-pioglitazone_Steady,change_Ch,diabetesMed_Yes,readmitted_<30,readmitted_>30
0,1,41,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,59,0,18,0,0,0,9,0,0,...,0,0,0,0,0,0,1,1,0,1
2,2,11,5,13,2,0,1,6,1,0,...,0,0,0,0,0,0,0,1,0,0
3,2,44,1,16,0,0,0,7,0,0,...,0,0,0,0,0,0,1,1,0,0
4,1,51,0,8,0,0,0,5,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,3,51,0,16,0,0,0,9,1,0,...,0,0,0,0,0,0,1,1,0,1
101762,5,33,3,18,0,0,1,9,1,0,...,0,0,0,0,0,0,0,1,0,0
101763,1,53,0,9,1,0,0,13,0,0,...,0,0,0,0,0,0,1,1,0,0
101764,10,45,2,21,0,0,1,9,0,0,...,0,0,0,0,0,0,1,1,0,0


# Checking if we really need all of the medication columns

Where any of the medications are 1 but diabetesMed is 0:

In [56]:
filtered_df = df[(df[['metformin_Down', 'metformin_Steady', 'metformin_Up',
                      'repaglinide_Down', 'repaglinide_Steady', 'repaglinide_Up',
                      'nateglinide_Down', 'nateglinide_Steady', 'nateglinide_Up',
                      'chlorpropamide_Down', 'chlorpropamide_Steady', 'chlorpropamide_Up',
                      'glimepiride_Down', 'glimepiride_Steady', 'glimepiride_Up',
                      'acetohexamide_Steady',
                      'glipizide_Down', 'glipizide_Steady', 'glipizide_Up',
                      'glyburide_Down', 'glyburide_Steady', 'glyburide_Up',
                      'tolbutamide_Steady',
                      'pioglitazone_Down', 'pioglitazone_Steady', 'pioglitazone_Up',
                      'rosiglitazone_Down', 'rosiglitazone_Steady', 'rosiglitazone_Up',
                      'acarbose_Down', 'acarbose_Steady', 'acarbose_Up',
                      'miglitol_Down', 'miglitol_Steady', 'miglitol_Up',
                      'troglitazone_Steady',
                      'tolazamide_Steady', 'tolazamide_Up',
                      'insulin_Down', 'insulin_Steady', 'insulin_Up',
                      'glyburide-metformin_Down', 'glyburide-metformin_Steady', 'glyburide-metformin_Up',
                      'glipizide-metformin_Steady',
                      'glimepiride-pioglitazone_Steady',
                      'metformin-pioglitazone_Steady', 'change_Ch']].any(axis=1)) & (df['diabetesMed_Yes'] == 0)]
filtered_df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,glyburide-metformin_Down,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-pioglitazone_Steady,change_Ch,diabetesMed_Yes,readmitted_<30,readmitted_>30


Where all of the medications are 0 but diabetesMed is 1:

In [57]:
filtered_df = df[(df[['metformin_Down', 'metformin_Steady', 'metformin_Up',
                      'repaglinide_Down', 'repaglinide_Steady', 'repaglinide_Up',
                      'nateglinide_Down', 'nateglinide_Steady', 'nateglinide_Up',
                      'chlorpropamide_Down', 'chlorpropamide_Steady', 'chlorpropamide_Up',
                      'glimepiride_Down', 'glimepiride_Steady', 'glimepiride_Up',
                      'acetohexamide_Steady',
                      'glipizide_Down', 'glipizide_Steady', 'glipizide_Up',
                      'glyburide_Down', 'glyburide_Steady', 'glyburide_Up',
                      'tolbutamide_Steady',
                      'pioglitazone_Down', 'pioglitazone_Steady', 'pioglitazone_Up',
                      'rosiglitazone_Down', 'rosiglitazone_Steady', 'rosiglitazone_Up',
                      'acarbose_Down', 'acarbose_Steady', 'acarbose_Up',
                      'miglitol_Down', 'miglitol_Steady', 'miglitol_Up',
                      'troglitazone_Steady',
                      'tolazamide_Steady', 'tolazamide_Up',
                      'insulin_Down', 'insulin_Steady', 'insulin_Up',
                      'glyburide-metformin_Down', 'glyburide-metformin_Steady', 'glyburide-metformin_Up',
                      'glipizide-metformin_Steady',
                      'glimepiride-pioglitazone_Steady',
                      'metformin-pioglitazone_Steady', 'change_Ch']].eq(0).all(axis=1)) & (df['diabetesMed_Yes'] == 1)]
filtered_df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,glyburide-metformin_Down,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-pioglitazone_Steady,change_Ch,diabetesMed_Yes,readmitted_<30,readmitted_>30


In [58]:
columns_to_drop = ['metformin_Steady', 'metformin_Up',
                   'repaglinide_Down', 'repaglinide_Steady', 'repaglinide_Up',
                   'nateglinide_Down', 'nateglinide_Steady', 'nateglinide_Up',
                   'chlorpropamide_Down', 'chlorpropamide_Steady', 'chlorpropamide_Up',
                   'glimepiride_Down', 'glimepiride_Steady', 'glimepiride_Up',
                   'acetohexamide_Steady',
                   'glipizide_Down', 'glipizide_Steady', 'glipizide_Up',
                   'glyburide_Down', 'glyburide_Steady', 'glyburide_Up',
                   'tolbutamide_Steady',
                   'pioglitazone_Down', 'pioglitazone_Steady', 'pioglitazone_Up',
                   'rosiglitazone_Down', 'rosiglitazone_Steady', 'rosiglitazone_Up',
                   'acarbose_Down', 'acarbose_Steady', 'acarbose_Up',
                   'miglitol_Down', 'miglitol_Steady', 'miglitol_Up',
                   'troglitazone_Steady',
                   'tolazamide_Steady', 'tolazamide_Up',
                   'insulin_Down', 'insulin_Steady', 'insulin_Up',
                   'glyburide-metformin_Down', 'glyburide-metformin_Steady', 'glyburide-metformin_Up',
                   'glipizide-metformin_Steady',
                   'glimepiride-pioglitazone_Steady',
                   'metformin-pioglitazone_Steady',
                   'change_Ch']

df.drop(columns=columns_to_drop, inplace=True)
df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,diabetesMed_Yes,readmitted_<30,readmitted_>30
0,1,41,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,59,0,18,0,0,0,9,0,0,...,0,0,0,0,0,0,0,1,0,1
2,2,11,5,13,2,0,1,6,1,0,...,0,0,0,0,0,0,0,1,0,0
3,2,44,1,16,0,0,0,7,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,51,0,8,0,0,0,5,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,3,51,0,16,0,0,0,9,1,0,...,0,0,0,1,0,0,0,1,0,1
101762,5,33,3,18,0,0,1,9,1,0,...,0,0,0,0,1,0,0,1,0,0
101763,1,53,0,9,1,0,0,13,0,0,...,0,0,0,1,0,0,0,1,0,0
101764,10,45,2,21,0,0,1,9,0,0,...,0,0,0,0,1,0,0,1,0,0


In [60]:
columns_except_diabetesMed_Yes = [col for col in df.columns if col != 'diabetesMed_Yes']

new_column_order = columns_except_diabetesMed_Yes + ['diabetesMed_Yes']

df = df[new_column_order]
df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
0,1,41,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,59,0,18,0,0,0,9,0,0,...,0,0,0,0,0,0,0,0,1,1
2,2,11,5,13,2,0,1,6,1,0,...,0,0,0,0,0,0,0,0,0,1
3,2,44,1,16,0,0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,51,0,8,0,0,0,5,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,3,51,0,16,0,0,0,9,1,0,...,0,0,0,1,0,0,0,0,1,1
101762,5,33,3,18,0,0,1,9,1,0,...,0,0,0,0,1,0,0,0,0,1
101763,1,53,0,9,1,0,0,13,0,0,...,0,0,0,1,0,0,0,0,0,1
101764,10,45,2,21,0,0,1,9,0,0,...,0,0,0,0,1,0,0,0,0,1


In [61]:
df.to_csv('diabetes130Hospitals10Years.csv', index=False)

In [3]:
import pandas as pd
df = pd.read_csv("diabetes130Hospitals10Years.csv")


,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
0,1,41,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,59,0,18,0,0,0,9,0,0,...,0,0,0,0,0,0,0,0,1,1
61,1,51,0,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
193,10,1,0,3,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
204,3,34,0,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98598,3,57,1,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
98674,8,88,6,30,0,0,1,9,0,0,...,0,0,0,0,0,0,0,1,0,1
98738,2,19,0,4,4,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,1
99432,3,44,0,10,0,0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df[((df['age_[0-10)'] == 1) | (df['age_[10-20)'] == 1)) & (df['diabetesMed_Yes'] == 1)]

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
1,3,59,0,18,0,0,0,9,0,0,...,0,0,0,0,0,0,0,0,1,1
61,1,51,0,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
193,10,1,0,3,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
204,3,34,0,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
220,2,48,1,10,0,0,1,5,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98552,3,2,0,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
98674,8,88,6,30,0,0,1,9,0,0,...,0,0,0,0,0,0,0,1,0,1
98738,2,19,0,4,4,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,1
99432,3,44,0,10,0,0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
df[((df['age_[0-10)'] == 1) | (df['age_[10-20)'] == 1)) & (df['diabetesMed_Yes'] == 0)]

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
0,1,41,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
539,2,22,1,14,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
858,2,61,0,4,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,1,0
1464,1,46,0,3,0,0,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1537,2,25,0,2,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96462,9,45,6,23,0,0,0,9,0,0,...,0,0,0,0,0,0,0,0,0,0
96605,1,33,0,1,3,1,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
96715,1,63,1,14,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
98311,3,1,1,12,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df[((df['age_[90-100)'] == 1) | (df['age_[90-100)'] == 1)) & (df['diabetesMed_Yes'] == 1)]

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
9,12,33,3,18,0,0,0,8,0,0,...,0,0,0,0,0,1,0,0,0,1
631,1,68,1,2,0,0,0,9,0,0,...,0,0,0,0,0,1,0,0,0,1
1028,4,53,0,11,0,0,1,7,0,0,...,0,0,0,0,0,1,0,0,0,1
1029,5,39,0,17,0,0,3,7,0,0,...,0,0,0,0,0,1,0,0,1,1
1437,4,57,0,13,0,0,1,9,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99206,9,58,0,22,1,0,4,9,0,0,...,0,0,0,0,0,1,0,1,0,1
99213,3,9,0,8,0,0,0,6,0,0,...,0,0,0,0,0,1,0,0,0,1
99256,2,59,2,16,0,0,1,9,0,0,...,0,0,0,0,0,1,0,0,0,1
99262,6,53,1,24,0,1,0,9,0,0,...,0,0,0,0,0,1,0,0,1,1


In [7]:
df[((df['age_[90-100)'] == 1) | (df['age_[90-100)'] == 1)) & (df['diabetesMed_Yes'] == 0)]

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
125,7,35,2,11,0,0,0,4,1,0,...,0,0,0,0,0,1,0,0,0,0
135,11,70,1,23,0,0,0,5,0,0,...,0,0,0,0,0,1,0,0,0,0
333,1,33,0,5,0,0,0,9,0,0,...,0,0,0,0,0,1,0,0,1,0
537,4,45,0,7,0,0,0,2,0,0,...,0,0,0,0,0,1,0,0,0,0
579,7,34,0,17,0,0,1,8,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99190,5,54,0,10,0,0,2,7,0,0,...,0,0,0,0,0,1,0,0,0,0
99315,2,1,1,10,0,0,1,9,0,0,...,0,0,0,0,0,1,0,1,0,0
99424,3,46,0,11,0,0,1,9,0,0,...,0,0,0,0,0,1,0,0,0,0
99426,6,60,0,20,0,1,0,9,1,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
df[((df['age_[90-100)'] == 0) & (df['age_[0-10)'] == 0) & (df['age_[10-20)'] == 0))]

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
2,2,11,5,13,2,0,1,6,1,0,...,0,0,0,0,0,0,0,0,0,1
3,2,44,1,16,0,0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,51,0,8,0,0,0,5,0,0,...,1,0,0,0,0,0,0,0,0,1
5,3,31,6,16,0,0,0,9,0,0,...,0,1,0,0,0,0,0,0,1,1
6,4,70,1,21,0,0,0,7,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99488,3,51,0,16,0,0,0,9,1,0,...,0,0,0,1,0,0,0,0,1,1
99489,5,33,3,18,0,0,1,9,1,0,...,0,0,0,0,1,0,0,0,0,1
99490,1,53,0,9,1,0,0,13,0,0,...,0,0,0,1,0,0,0,0,0,1
99491,10,45,2,21,0,0,1,9,0,0,...,0,0,0,0,1,0,0,0,0,1


In [10]:
df[((df['age_[90-100)'] == 0) & (df['age_[0-10)'] == 0) & (df['age_[10-20)'] == 0))&(df['diabetesMed_Yes'] == 1)]

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
2,2,11,5,13,2,0,1,6,1,0,...,0,0,0,0,0,0,0,0,0,1
3,2,44,1,16,0,0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,51,0,8,0,0,0,5,0,0,...,1,0,0,0,0,0,0,0,0,1
5,3,31,6,16,0,0,0,9,0,0,...,0,1,0,0,0,0,0,0,1,1
6,4,70,1,21,0,0,0,7,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99487,6,45,1,25,3,1,2,9,1,0,...,0,0,1,0,0,0,0,0,1,1
99488,3,51,0,16,0,0,0,9,1,0,...,0,0,0,1,0,0,0,0,1,1
99489,5,33,3,18,0,0,1,9,1,0,...,0,0,0,0,1,0,0,0,0,1
99490,1,53,0,9,1,0,0,13,0,0,...,0,0,0,1,0,0,0,0,0,1


In [11]:
df[((df['age_[90-100)'] == 0) & (df['age_[0-10)'] == 0) & (df['age_[10-20)'] == 0))&(df['diabetesMed_Yes'] == 0)]

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,...,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),metformin_Down,readmitted_<30,readmitted_>30,diabetesMed_Yes
19,2,47,0,12,0,0,0,8,1,0,...,0,0,0,1,0,0,0,0,0,0
20,11,42,2,19,0,0,0,8,0,0,...,0,0,0,0,1,0,0,0,1,0
47,4,40,1,14,0,0,0,9,1,0,...,0,1,0,0,0,0,0,1,0,0
65,7,75,2,22,0,0,0,7,0,0,...,0,0,0,1,0,0,0,1,0,0
76,3,52,0,13,0,0,2,9,1,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99427,4,44,0,17,0,0,2,9,0,0,...,0,0,0,0,1,0,0,0,0,0
99440,7,60,0,17,0,1,1,9,1,0,...,0,0,0,0,1,0,0,0,1,0
99458,3,57,4,11,0,0,1,9,1,0,...,0,0,0,1,0,0,0,0,0,0
99471,1,49,0,12,0,0,0,9,0,0,...,0,0,0,0,1,0,0,0,0,0
